In [1]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter serverextension enable voila --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling: voila
- Writing config: /home/awpi/.pyenv/versions/3.11.3/envs/tt_client/etc/jupyter
    - Validating...
      voila 0.4.0 OK


In [2]:
import warnings
#warnings.filterwarnings("ignore")

import ipywidgets as widgets
from IPython.display import display, clear_output

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from client.client import TraningTrackingClient, ClientException
from shared_models.schemas import Exercise, Skill

In [5]:
client = TraningTrackingClient("http://192.168.178.26:8080")
client.assert_api_health()

In [24]:
id_input = widgets.Text(
    description='id:',
    disabled=False   
)
descrption_input = widgets.Textarea(
    description='desc:',
    disabled=False   
)
domain_input=widgets.Dropdown(
    options=['gym', 'running', 'drums', 'guitar'],
    description='domain:',
    value='drums',
    disabled=False
)
insert_button = widgets.Button(description="insert skill")
delete_button = widgets.Button(description="delete skill")
last_inserted = widgets.Output()

def insert_skill(event):
    with last_inserted:
        clear_output()

        skill = Skill(id=id_input.value, description=descrption_input.value, domain=domain_input.value)
        try:
            client.post_skill(skill)
            print(f"inserted {skill}")

        except ClientException as e:
            print(e)
            

def delete_skill(event):    
    with last_inserted:
        clear_output()

        try:
            client.delete_skill(id_input.value)
            print(f"delteted skill w/ id={id_input.value}")

        except ClientException as e:
            print(e)
        
insert_button.on_click(insert_skill)
delete_button.on_click(delete_skill)
        
new_skill_box = widgets.VBox([widgets.HBox([id_input, domain_input, descrption_input, insert_button, delete_button]), last_inserted])
new_skill_box

In [22]:
output_get_skill = widgets.Output()
button_get_skill = widgets.Button(description="get skills")

def on_get_skill_click(event):
    with output_get_skill:
        clear_output()
        for skill in client.get_skills():
            print(f"id={skill.id}")

button_get_skill.on_click(on_get_skill_click)

get_skill_box = widgets.VBox([button_get_skill, output_get_skill])
display(get_skill_box)

In [23]:
skill_ids = [s.id for s in client.get_skills()]

id_input = widgets.Text(
    description='id:',
    disabled=False   
)
variation_input = widgets.Text(
    description='variation:',
    disabled=False   
)
descrption_input = widgets.Textarea(
    description='desc:',
    disabled=False   
)
domain_input=widgets.Dropdown(
    options=['gym', 'running', 'drums', 'guitar'],
    description='domain:',
    value='drums',
    disabled=False
)
skills_input=widgets.SelectMultiple(
    options=skill_ids,
    description='skills:',
    disabled=False
)
insert_button = widgets.Button(description="insert exercise")
delete_button = widgets.Button(description="delete exercise")
last_inserted = widgets.Output()

def set_skills(event):
    skills_input.options = [s.id for s in client.get_skills(domain=domain_input.value)]

set_skills(None)
    
def insert_exercise(event):
    with last_inserted:
        clear_output()

        exercise = Exercise(id=id_input.value, 
                            variation=variation_input.value, 
                            domain=domain_input.value, 
                            description=descrption_input.value,
                            skills=[Skill(id=id, domain=domain_input.value) for id in skills_input.value]
                           )
        try:
            client.post_exercise(exercise)
            print(f"inserted {exercise}")

        except ClientException as e:
            print(e)
            

def delete_exercise(event):
    with last_inserted:
        clear_output()

        try:
            client.delete_exercise(id=id_input.value, variation=variation_input.value)
            print(f"delteted skill w/ id={id_input.value}/{variation_input.value}")

        except ClientException as e:
            print(e)
        
insert_button.on_click(insert_exercise)
delete_button.on_click(delete_exercise)
domain_input.observe(set_skills)
        
new_skill_box = widgets.VBox([widgets.HBox([id_input, variation_input, domain_input]), 
                              widgets.HBox([descrption_input, skills_input, insert_button, delete_button]), last_inserted])
new_skill_box

In [10]:
output_get_exercise = widgets.Output()
button_get_exercise = widgets.Button(description="get exercises")

def on_get_exercise_click(event):
    with output_get_exercise:
        clear_output()
        for exercise in client.get_exercises():
            print(exercise.description)
            print(f"id={exercise.id}, variation={exercise.variation}, skills: {','.join([s.id for s in exercise.skills])}")

button_get_exercise.on_click(on_get_exercise_click)

get_exercise_box = widgets.VBox([button_get_exercise, output_get_exercise])
display(get_exercise_box)